In [7]:
from glob import glob
import pandas as pd
import numpy as np
from scipy import stats  
import matplotlib.pyplot as plt
import japanize_matplotlib

In [21]:
files = glob("../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎/*")
for i, f in enumerate(files):
    print(i, f)


0 ../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎\模擬問題問題1_8月売上データ.xlsx
1 ../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎\模擬問題問題2_中華まんとお茶の嗜好調査.xlsx
2 ../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎\模擬問題問題3_コピー機のトナー交換日数.xlsx
3 ../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎\模擬問題問題4_ゲームキャラクターのステータス.xlsx
4 ../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎\模擬問題問題6_基礎知識テストのデータ.xlsx
5 ../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎\模擬問題問題7_3店舗の月別売上高.xlsx
6 ../csv/データアナリティクス基礎/模擬問題_データアナリティクス基礎\模擬問題問題8_自由記述データ＆形態素解析データ.xlsx


In [22]:
df = pd.read_excel(files[0])
print(df.columns)
df.columns = ['id', 'dept', 'name', 'industry', 'sales']
df.info()
df.head()

Index(['データ番号', '課名', '担当者名', '業種', '8月\n売上金額（円）'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        191 non-null    int64 
 1   dept      191 non-null    object
 2   name      191 non-null    object
 3   industry  191 non-null    object
 4   sales     191 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 7.6+ KB


,id,dept,name,industry,sales
0,1,営業１課,T.N.,給食業,1689621
1,2,営業１課,T.N.,ミニスーパー,204516
2,3,営業１課,T.N.,ミニスーパー,37260
3,4,営業１課,T.N.,スーパー,2621256
4,5,営業１課,T.N.,レストラン,529894


In [23]:
df['industry'] = df['industry'].str.replace('\u3000', '')

In [24]:
print(f'業種の数 : {df["industry"].nunique()}')

業種の数 : 9


In [25]:
df.query('dept == "営業２課"').groupby('industry').sum(numeric_only=True).sort_values(by='sales', ascending=False)

,id,sales
industry,,
食料品店,1465,14087641
レストラン,2426,12868977
スーパー,1224,12157158
食堂業,1443,10198263
ミニスーパー,400,4516493
食料品卸業,91,998004
給食業,367,986747
百貨店,316,919834
コンビニ,224,393971


In [26]:
df_sales = df.groupby('industry').sum(numeric_only=True)
df_sales = df_sales.drop(['id'], axis=1)
df_sales['ratio'] = df_sales['sales'] / df_sales['sales'].sum()
display(df_sales)
top3 =  df_sales.sort_values(by='ratio', ascending=False)[:3]['ratio'].sum() * 100
print(f'上位3つの売上割合 : {top3:.01f}')

,sales,ratio
industry,,
コンビニ,4914967,0.028480
スーパー,53984282,0.312819
ミニスーパー,16682430,0.096669
レストラン,18395880,0.106597
百貨店,2137664,0.012387
給食業,6327296,0.036664
食堂業,13095061,0.075881
食料品卸業,8762609,0.050776
食料品店,48273295,0.279726


上位3つの売上割合 : 69.9


In [27]:
df_cross = df.pivot_table(index='dept', columns='industry', values='sales', aggfunc='sum').sort_values(by='スーパー', ascending=False)
print('営業１課')
df_cross

営業１課


industry,コンビニ,スーパー,ミニスーパー,レストラン,百貨店,給食業,食堂業,食料品卸業,食料品店
dept,,,,,,,,,
営業１課,3898248,35770882,8118126,2417600,368600,3482332,2274679,788423,16265972
営業２課,393971,12157158,4516493,12868977,919834,986747,10198263,998004,14087641
営業３課,622748,6056242,4047811,3109303,849230,1858217,622119,6976182,17919682


特化係数 列集計

In [28]:
df_spec = df_cross.copy()
df_spec['total']
df_spec_column = df_spec / df_spec.sum()
# print(df_spec_column.sum())
df_spec_column

KeyError: 'total'

特化係数 行集計

In [ ]:
df_spec = df_cross.copy().T
df_spec_index = df_spec / df_spec.sum()
print(df_spec_index.sum())
df_spec_index.T

dept
営業１課    1.0
営業２課    1.0
営業３課    1.0
dtype: float64


industry,コンビニ,スーパー,ミニスーパー,レストラン,百貨店,給食業,食堂業,食料品卸業,食料品店
dept,,,,,,,,,
営業１課,0.053121,0.487442,0.110624,0.032944,0.005023,0.047453,0.030997,0.010744,0.221653
営業２課,0.006896,0.212809,0.079060,0.225269,0.016102,0.017273,0.178519,0.017470,0.246602
営業３課,0.014806,0.143985,0.096235,0.073923,0.020190,0.044179,0.014791,0.165857,0.426035


In [ ]:
df_spec_index = df_cross.copy()
# df_spec_index.loc['total'] = df_spec_index.sum()
df_spec_index['total'] = df_spec_index.apply(lambda x: sum(x), axis=1)
display(df_spec_index)
df_spec_index = df_spec_index / df_spec_index.sum()
display(df_spec_index)
df_spec_index.apply(lambda x: x / df_spec_index['total'])


industry,コンビニ,スーパー,ミニスーパー,レストラン,百貨店,給食業,食堂業,食料品卸業,食料品店,total
dept,,,,,,,,,,
営業１課,3898248,35770882,8118126,2417600,368600,3482332,2274679,788423,16265972,73384862
営業２課,393971,12157158,4516493,12868977,919834,986747,10198263,998004,14087641,57127088
営業３課,622748,6056242,4047811,3109303,849230,1858217,622119,6976182,17919682,42061534


industry,コンビニ,スーパー,ミニスーパー,レストラン,百貨店,給食業,食堂業,食料品卸業,食料品店,total
dept,,,,,,,,,,
営業１課,0.793138,0.662617,0.486627,0.131421,0.172431,0.550367,0.173705,0.089976,0.336956,0.425238
営業２課,0.080157,0.225198,0.270734,0.699558,0.430299,0.155951,0.778787,0.113893,0.291831,0.331031
営業３課,0.126704,0.112185,0.242639,0.169022,0.397270,0.293683,0.047508,0.796131,0.371213,0.243731


industry,コンビニ,スーパー,ミニスーパー,レストラン,百貨店,給食業,食堂業,食料品卸業,食料品店,total
dept,,,,,,,,,,
営業１課,1.865162,1.558224,1.144364,0.309052,0.405493,1.294254,0.408489,0.211589,0.792393,1.0
営業２課,0.242145,0.680294,0.817851,2.113272,1.299876,0.471107,2.352614,0.344057,0.881583,1.0
営業３課,0.519853,0.460283,0.995520,0.693476,1.629952,1.204945,0.194919,3.266430,1.523044,1.0


In [ ]:
df_spec_column = df_cross.copy()
df_spec_column.loc['total'] = df_spec_column.sum()
display(df_spec_column)


industry,コンビニ,スーパー,ミニスーパー,レストラン,百貨店,給食業,食堂業,食料品卸業,食料品店
dept,,,,,,,,,
営業１課,3898248,35770882,8118126,2417600,368600,3482332,2274679,788423,16265972
営業２課,393971,12157158,4516493,12868977,919834,986747,10198263,998004,14087641
営業３課,622748,6056242,4047811,3109303,849230,1858217,622119,6976182,17919682
total,4914967,53984282,16682430,18395880,2137664,6327296,13095061,8762609,48273295


In [ ]:
tmp = []
for index in df_spec_column.index:
    tmp.append(df_spec_column.loc[index] / df_spec_column.loc['total'])
pd.DataFrame(tmp)

industry,コンビニ,スーパー,ミニスーパー,レストラン,百貨店,給食業,食堂業,食料品卸業,食料品店
Unnamed 0,0.793138,0.662617,0.486627,0.131421,0.172431,0.550367,0.173705,0.089976,0.336956
Unnamed 1,0.080157,0.225198,0.270734,0.699558,0.430299,0.155951,0.778787,0.113893,0.291831
Unnamed 2,0.126704,0.112185,0.242639,0.169022,0.397270,0.293683,0.047508,0.796131,0.371213
total,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
